In [2]:
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from peft import LoraConfig, get_peft_model
from datasets import load_dataset
from trl import SFTTrainer  # Supervised Fine-Tuning Trainer

# Set our name for the finetune to be saved &/ uploaded to
finetune_name = "SFT-Dolly-HistoryDataset"
finetune_tags = ["SFT", "Dolly"]

# Load model and tokenizer
model_name = "databricks/dolly-v2-3b"
dataset_name = "gauri-sharan/history-class8-dataset"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,  # Use 8-bit quantization if needed
    device_map="auto"
)

# Load dataset
dataset = load_dataset(dataset_name)

# Inspect the dataset structure
print(dataset["train"][0])  # Check the structure of the first example

# Tokenize dataset
def tokenize_function(examples):
    # Adjust the fields based on the dataset structure
    # For example, if the dataset has "prompt" and "completion" fields:
    text_samples = [p + "\n" + c for p, c in zip(examples["input"], examples["output"])]
    return tokenizer(text_samples, truncation=True, padding="max_length", max_length=512)

# Apply tokenization to the dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Split dataset into training and testing manually
split_dataset = tokenized_dataset["train"].train_test_split(test_size=0.1)

# Assign train and test datasets
train_dataset = split_dataset["train"]
test_dataset = split_dataset["test"]

# LoRA Configuration
lora_config = LoraConfig(
    r=8,  # LoRA rank
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["query_key_value"]  # Correct module for Dolly-v2-3B
)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# Training arguments
training_args = TrainingArguments(
    output_dir="./dolly-finetuned-2",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    evaluation_strategy="steps",
    eval_steps=500,
    save_strategy="steps",
    save_steps=1000,
    logging_dir="./logs",
    logging_steps=10,
    learning_rate=2e-5,
    weight_decay=0.01,
    num_train_epochs=3,
    report_to="none",  # Disable reporting to wandb
    push_to_hub=True,
    hub_model_id=finetune_name,
)

# Initialize the SFT Trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    args=training_args,
)

# Train the model
trainer.train()

# Save model
model.save_pretrained("./dolly-finetuned-2")
tokenizer.save_pretrained("./dolly-finetuned-2")


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


{'instruction': 'Summarize the following historical passage:', 'input': '59\nreinforcements from England, passed new laws so\nthat the rebels could be convicted with ease, and then\nmoved into the storm centres of the revolt. Delhi was\nrecaptured from the rebel forces in September 1857. The\nlast Mughal emperor, Bahadur Shah Zafar was tried in\ncourt and sentenced to life imprisonment. He and his\nwife Begum Zinat Mahal were sent to prison in Rangoon\nin October 1858. Bahadur Shah Zafar died in the Rangoon\njail in November 1862.\nThe recapture of Delhi, however, did not mean that the\nrebellion died down after that. People continued to resist\nand battle the British. The British had to fight for two\nyears to suppress the massive forces of popular rebellion.\nLucknow was taken in  March 1858. Rani Lakshmibai\nwas defeated and killed in June 1858. A similar fate\nawaited Rani Avantibai, who after initial victory in\nKheri, chose to embrace death when surrounded by the\nBritish on all 

/raid/home/mukesh/.local/lib/python3.10/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_34106/3705277963.py:78: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(
/raid/home/mukesh/.local/lib/python3.10/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Converting train dataset to ChatML:   0%|          | 0/108 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/108 [00:00<?, ? examples/s]

...

Applying chat template to eval dataset:   0%|          | 0/12 [00:00<?, ? examples/s]

Applying chat template to eval dataset:   0%|          | 0/12 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModel`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss,Validation Loss


('./dolly-finetuned-2/tokenizer_config.json',
 './dolly-finetuned-2/special_tokens_map.json',
 './dolly-finetuned-2/tokenizer.json')

In [3]:
# Save the model
trainer.save_model(f"./{finetune_name}")

# Push to hub
trainer.push_to_hub(tags=finetune_tags)

CommitInfo(commit_url='https://huggingface.co/gauri-sharan/SFT-Dolly-HistoryDataset/commit/24b665606eb7f1e62e6e8aea6ca1d08ef07dd4a4', commit_message='End of training', commit_description='', oid='24b665606eb7f1e62e6e8aea6ca1d08ef07dd4a4', pr_url=None, repo_url=RepoUrl('https://huggingface.co/gauri-sharan/SFT-Dolly-HistoryDataset', endpoint='https://huggingface.co', repo_type='model', repo_id='gauri-sharan/SFT-Dolly-HistoryDataset'), pr_revision=None, pr_num=None)

In [4]:
train_loss_history = trainer.state.log_history  # Get loss logs
for log in train_loss_history:
    if "loss" in log:
        print(f"Step {log.get('step', 'N/A')}: Loss = {log['loss']}")

Step 10: Loss = 2.6519
Step 20: Loss = 2.5915
Step 30: Loss = 2.6017
Step 40: Loss = 2.5678
Step 50: Loss = 2.5455
Step 60: Loss = 2.6074
Step 70: Loss = 2.5862
Step 80: Loss = 2.5285


In [1]:
from huggingface_hub import login

login()